1. Thu thập các đường dẫn thuộc mỗi chuyên mục từ trang web https://vnexpress.net/

- Viết hàm lấy tất cả các đường dẫn của bài báo trong trang web

In [1]:
# Gọi thư viện
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import re

# Đường dẫn đến trang web gốc:
URL = 'https://vnexpress.net/'

def _get_links(url):
    links = [] 
    html = urlopen(url).read()  
    soup = BeautifulSoup(html, 'html.parser') 

    # Lấy tất cả đường dẫn trong trang web hiện tại
    urls = set(re.findall(r'https://vnexpress.net/[^\s"]+\.html', str(soup)))  

    for url in urls:
        # Chỉ lấy đường dẫn là trang web nhiều bài báo
        if url != URL and url.startswith(URL) and not url.startswith(URL + '/tac-gia'): 
            links.append(url)

    return links


- Kết quả trả về của hàm với đường dẫn gốc của VNexpress

In [3]:
_get_links(URL)

['https://vnexpress.net/no-lo-hoi-lam-sap-tuong-5-can-nha-o-tp-hcm-4804560.html',
 'https://vnexpress.net/nguoi-nuoi-gap-kho-do-trung-quoc-chuong-tom-hum-co-nho-4803131.html',
 'https://vnexpress.net/con-sot-tui-mu-cua-gioi-tre-viet-4802141.html',
 'https://vnexpress.net/ap-luc-bua-vay-nganh-dich-vu-phong-gym-4803994.html',
 'https://vnexpress.net/can-thu-ke-lai-32-gio-troi-dat-tren-bien-4804514.html',
 'https://vnexpress.net/unifil-luc-luong-gin-giu-hoa-binh-giua-lan-dan-israel-4804178.html',
 'https://vnexpress.net/dinh-thu-hoang-a-tuong-sau-trung-tu-4804529.html',
 'https://vnexpress.net/ten-lua-manh-nhat-the-gioi-starship-co-gi-dac-biet-4804521.html',
 'https://vnexpress.net/gia-cuoc-5g-the-nao-so-voi-4g-4804360.html',
 'https://vnexpress.net/de-nghi-lam-ro-muc-dau-tu-hon-67-ty-usd-duong-sat-toc-do-cao-bac-nam-4804562.html',
 'https://vnexpress.net/luong-giao-vien-bao-nhieu-la-du-4804505.html',
 'https://vnexpress.net/nhan-su-it-chat-vat-tim-viec-giu-vi-tri-4804546.html',
 'https:/

- Lấy đường dẫn của các bài toán thuộc từng chuyên mục

In [4]:
# Danh sách các chuyên mục
categories = [
    'thoi-su', 'kinh-doanh', 'khoa-hoc', 'giai-tri',
    'the-thao', 'phap-luat', 'giao-duc', 'suc-khoe',
    'doi-song', 'du-lich', 'so-hoa', 'xe'
]

# Danh sách các đường dẫn đến các bài báo của mỗi chuyên mục
article_url = {}

for category in categories:
    article_url[category] = []

    # Lấy 5 trang trong mỗi chuyên mục
    for i in range(1, 3):
        url = '{}/{}-p{}'.format(URL, category, str(i))
        article_url[category] += _get_links(url)

    # Loại bỏ đường dẫn trùng nhau
    article_url[category] = list(set(article_url[category]))

    # Hiển thị số đường dẫn thu thập được trong chuyên mục
    print(category, len(article_url[category]))



thoi-su 27
kinh-doanh 27
khoa-hoc 27
giai-tri 27


URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [11]:
# Lấy riêng các đường dẫn trong chuyên mục theo góc nhìn từng mục con
goc_nhin = [
    'binh-luan-nhieu', 'chinh-tri-chinh-sach', 'y-te-su-khoe',
    'kinh-doanh-quan-tri', 'giao-duc-tri-thuc', 'moi-truong',
    'van-hoa-loi-song', 'covid-19'
]

article_url['goc-nhin'] = []

for sub_cate in goc_nhin:
    url = '{}/{}/{}'.format(URL, 'goc-nhin', sub_cate)
    article_url['goc-nhin'] += _get_links(url)

# Loại bỏ đường dẫn trùng nhau trong chuyên mục 'goc-nhin'
article_url['goc-nhin'] = list(set(article_url['goc-nhin']))
print('goc-nhin', len(article_url['goc-nhin']))

goc-nhin 27


2. Lấy nội dung của bài báo ứng với từng đường dẫn cụ thể và lưu vào một file trong folder chuyên mục tương ứng.

- Viết hàm lấy nội dung của bài báo ứng với đường dẫn cụ thể

In [15]:
def _get_content(url):
    # Nội dung bài báo
    content = ''
    
    # Mở URL và đọc nội dung trang
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    # Lấy phần chứa nội dung bài báo
    div_content = soup.select('.page-detail .container')
    if len(div_content) > 0:
        div_content = div_content[0]

        # Lấy phần mô tả
        description = div_content.find_all('p', {'class': 'description'})
        if len(description) > 0:
            text_description = description[-1].get_text()
            location = description[-1].find('span', {'class': 'location-stamp'})
            if location is not None:
                content = text_description[len(location.get_text()):]
            else:
                content = text_description   

        # Lấy phần chứa nội dung chi tiết bài báo
        detail = div_content.find('article', {'class': 'fck_detail'})
        if detail is not None:
            p_normal = detail.find_all('p', {'class': 'Normal'})
            if len(p_normal) > 0:
                for p in p_normal:
                    p_text = p.get_text()
                    # Kiểm tra đoạn văn bản không rỗng, không bắt đầu bằng dấu '>>' và kết thúc bằng các ký tự hợp lệ
                    if p_text != '' and not p_text.startswith('>>') and p_text[-1] in ['.', '!', '?']:
                        content += ' ' + p_text

    # Xóa ký tự ngắt dòng và trả về
    return re.sub(r'\n', '', content)

+ Kết quả trả về của hàm với đường dẫn dầu tiên trong chuyên mục số hóa

In [17]:
_get_content(article_url['giao-duc'][0])

KeyError: 'giao-duc'

- Lưu nội dung các bài báo thành các file nằm trong folder chuyên mục tương ứng

In [ ]:
import os

# Danh sách tất cả các câu đã làm sạch của tất cả các bài viết
sentence = []
#Chữ cái in hoa cảu tiếng việt
uppercase = "AĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXY"

for category in article_url.keys():
    # Tạo folder chuyên mục để chứa các file bài viết
    current_path = 'data/categories/' + category + '/'
    os.mkdir(current_path)

# Lưu các bài viết thành từng file
    count = 0
    for i in range(len(article_url[category])):
        content = _get_content(article_url[category][i])
        if content != '':
            count += 1
            f = open('{}/{}_{}.txt'.format(current_path, category, str(count)), 'w', encoding='utf-8') 
            # Chèn khoảng cách vào sau các dấu câu (., ?, !) nếu không có khoảng trắng hoặc dấu ngoặc kép ngay sau đó
            lines = re.sub(r'(?<=[.?!])(?=[\'"{}])'.format(uppercase), ' ', content)

            # Tách bài viết thành câu (giả sử câu kết thúc bằng ., !, ? và câu mới bắt đầu bằng chữ cái in hoa hoặc chữ số)
            lines = re.split(r'(?<=[.?!])\s*(?=[\'"{}0-9])'.format(uppercase), content)


            # Ghi từng câu trong vào viết vào file
            for line in lines:
                f.write((line + '\n'))
                sentence.append(line)
            f.close
        # Sổ bài viết trong chuyên mục    
        print(category, count)

        sentence = list(set(sentence))
        print("Tổng số câu được tách: ", len(sentence))

+ 10 câu dầu tiên tách được

In [ ]:
sentence[:10]

<strong>3. Lưu tất cả các câu vào file ngữ liệu </strong>

In [ ]:
# File ngữ liệu
file_corpus = open('data/corpus.txt', 'w', encoding='utf-8')

count = 0
for sent in sentence:
    # Xóa ký tự thừa
    sent = re.sub(r'\u200b', '', sent)

    file_corpus.write(sent.strip() + '\n')
    count +=1
file_corpus.close()

print('Số lượng câu trong file ngữ liệu: ', count)